# creat model

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer 

creat embedding layer

In [2]:
class Embedding(Layer):
  def __init__(self):
    super(Embedding,self).__init__()
    self.c1=tf.keras.layers.Conv2D(64,(10,10),activation='relu',input_shape=(100,100,3))
    self.c2=tf.keras.layers.Conv2D(128,(7,7),activation='relu')
    self.c3=tf.keras.layers.Conv2D(128,(4,4),activation='relu')
    self.c4=tf.keras.layers.Conv2D(256,(4,4),activation='relu')
    self.pool=tf.keras.layers.MaxPooling2D(64,(2,2),padding='same')
    self.flatt=tf.keras.layers.Flatten()
    self.dense=tf.keras.layers.Dense(4096,activation='sigmoid')
  
  def call(self,input_tensor):
    x=self.c1(input_tensor)
    x=self.pool(x)
    x=self.c2(x)
    x=self.pool(x)
    x=self.c3(x)
    x=self.pool(x)
    x=self.c4(x)
    x=self.flatt(x)
    return  self.dense(x)

In [3]:
embedding=Embedding()
embedding.build(input_shape=(100,100,3))

creat distance layer

In [4]:
class L1Dist(Layer):
    def __init__(self):
      super(L1Dist,self).__init__()
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [5]:
L1dist=L1Dist()

creat siames model

In [6]:
input_anc=tf.keras.layers.Input(shape=(100,100,3))
input_2=tf.keras.layers.Input(shape=(100,100,3))

emb_1=embedding(input_anc)
emb_2=embedding(input_2)
distance=L1dist(emb_1,emb_2)
classifier=tf.keras.layers.Dense(1,activation='sigmoid')(distance)

siamese_model=tf.keras.models.Model([input_anc,input_2],classifier)

In [7]:
siamese_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Embedding)          (None, 4096)         38960448    ['input_1[0][0]',                
                                                                  'input_2[0][0]']            

# load and prepeocess data

In [8]:
import cv2 
import numpy as np
import random 
import matplotlib.pyplot as plt 
import os 

In [9]:
%cd '/content/drive/MyDrive/faceR'

/content/drive/MyDrive/faceR


In [10]:
Neg_path=os.path.join('data','negative')
Pos_path=os.path.join('data','positive')
Anc_path=os.path.join('data','anchor')

In [ ]:
os.makedirs(Neg_path)
os.makedirs(Pos_path)
os.makedirs(Anc_path)

In [ ]:
for directory in os.listdir('lfw'):
  for files in os.listdir(os.path.join('lfw',directory)):
    Ex_path=os.path.join('lfw',directory,files)
    New_path= os.path.join(Neg_path, files)
    os.replace(Ex_path,New_path)

In [ ]:
import uuid

In [ ]:
cam=cv2.VideoCapture(0)
while True:
    _,frame=cam.read()
    frame=cv2.flip(frame,1)
    frame = frame[120:120+250,200:200+250, :]
    if cv2.waitKey(1)& 0XFF ==ord('p'):
        image_name=os.path.join(Pos_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(image_name,frame)
    
    if cv2.waitKey(1)& 0XFF ==ord('a'):
        image_name=os.path.join(Anc_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(image_name,frame)
    
    cv2.imshow('fram',frame)
    if cv2.waitKey(1)& 0XFF ==27:
        break

creat helper fuction 

In [11]:
def normalize(image):
  image=(image/127.5)-1
  return image
def resize(image,height,width):
  image=tf.image.resize(image,(height,width),method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
  return image

def load_image(image_path):
  image=tf.io.read_file(image_path)
  image=tf.image.decode_jpeg(image,3)
  image=tf.cast(image,tf.float32)
  image=resize(image,100,100)
  image=normalize(image)
  return image 


In [12]:
def load_anchor_positive_with_label(image_path,positive_path):
  anchor=load_image(image_path)
  positive=load_image(positive_path)
  label=tf.ones(1)
  return anchor , positive , label

def load_anchor_negative_with_label(image_path,negative_path):
  anchor=load_image(image_path)
  negative=load_image(negative_path)
  label=tf.zeros(1)
  return anchor , negative , label

In [13]:
anchor=tf.data.Dataset.list_files(Anc_path+'/*.jpg').take(300)
positive=tf.data.Dataset.list_files(Pos_path+'/*.jpg').take(300)
negative=tf.data.Dataset.list_files(Neg_path+'/*.jpg').take(300)

In [14]:
anc_pos=tf.data.Dataset.zip((anchor,positive))
anc_neg=tf.data.Dataset.zip((anchor,negative))

In [15]:
anc_pos_data=anc_pos.map(load_anchor_positive_with_label)
anc_neg_data=anc_neg.map(load_anchor_negative_with_label)

In [16]:
data = anc_pos_data.concatenate(anc_neg_data)

In [17]:
data=data.shuffle(1000)
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)

In [18]:
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)

## train model 

In [19]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4) 

checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [20]:
def train_step(batch):
    
    
    with tf.GradientTape() as tape:     
        X = batch[:2]
        y = batch[2]
        
        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)
    print(loss)
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

In [21]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
       
        recall=tf.keras.metrics.Recall()
        precision=tf.keras.metrics.Precision()
        
        
        for idx, batch in enumerate(data):
           
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            recall.update_state(batch[2], yhat)
            precision.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), recall.result().numpy(), precision.result().numpy())
        
      
        if epoch % 4 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [22]:
EPOCHS = 8
train(train_data, EPOCHS)

tf.Tensor(0.1052363, shape=(), dtype=float32)
27/27 [==============================] - 103s 742ms/step
0.0005638659 1.0 0.9861751


In [23]:
siamese_model.save('my_model.h5')